In [1]:
import cv2
import mediapipe as mp
import csv
from datetime import datetime, timedelta

# CSV 파일에 헤더를 추가
with open('all.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Mouth Brightness', 'Mouth Openness', 'Label'])

# Mediapipe의 Face Mesh 모델 로드
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# 웹캠에서 영상을 받아오는 VideoCapture 객체 생성
cap = cv2.VideoCapture(0)

start_time = datetime.now()
elapsed_time = timedelta(seconds=0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("비디오 캡처 실패, 종료합니다.")
        break
    
    # 프레임을 BGR에서 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # 얼굴 검출 및 랜드마크 추출
    results = face_mesh.process(rgb_frame)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # 입 주변의 랜드마크를 추출하여 입안 명암도 계산
            mouth_landmarks = face_landmarks.landmark[12:20]
            mouth_area = frame.copy()
            for landmark in mouth_landmarks:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(mouth_area, (x, y), 2, (255, 0, 0), -1)
            mouth_brightness = cv2.cvtColor(mouth_area, cv2.COLOR_BGR2GRAY).mean()
            
            # 입 벌림 정도를 계산
            upper_lip_bottom = (face_landmarks.landmark[13].x, face_landmarks.landmark[13].y)
            lower_lip_top = (face_landmarks.landmark[14].x, face_landmarks.landmark[14].y)
            mouth_openness = lower_lip_top[1] - upper_lip_bottom[1]
            
            # 경과 시간이 0.2초 이상일 때만 CSV 파일에 추가
            if elapsed_time.total_seconds() >= 0.2:
                # 시간 단위로 입안 명암도와 입 벌림 정도를 CSV 파일에 추가
                with open('all.csv', mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([mouth_brightness, mouth_openness, 4])
                
                # 경과 시간 초기화
                elapsed_time = timedelta(seconds=0)
            else:
                # 경과 시간 누적
                elapsed_time += datetime.now() - start_time
                start_time = datetime.now()
            
            # 입 벌림 정도와 입안 명암도를 프레임에 표시
            cv2.putText(frame, f"Mouth Openness: {mouth_openness:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Mouth Brightness: {mouth_brightness:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            

    # 영상 출력
    cv2.imshow('Face Mesh', frame)
    
    # 종료 키 (q) 입력 시 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()
